**Importing**

In [ ]:
import numpy as np

import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

from  matplotlib import pyplot as plt

**Create your model**

In [ ]:
def create_model(input_shape, pool_size):
    # TODO: Create the Model sequentially
    model = Sequential()
    #Batch Normalization --Layer 1
    model.add(BatchNormalization(input_shape=input_shape))

    #Convolution + Relu --Layer 2
    model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))

    #Convolution + Relu --Layer 3
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))

    #Pooling --Layer 4
    model.add(MaxPooling2D(pool_size=pool_size))

    #Convolution + Relu (includes dropout) --Layer 5
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Convolution + Relu (includes dropout) --Layer 6
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Convolution + Relu (includes dropout) --Layer 7
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Pooling --Layer 8
    model.add(MaxPooling2D(pool_size=pool_size))

    #Convolution + Relu (includes dropout) --Layer 9
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Convolution + Relu (includes dropout) --Layer 10
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Pooling --Layer 11
    model.add(MaxPooling2D(pool_size=pool_size))

    #Upsampling --Layer 12
    model.add(UpSampling2D(size=pool_size))

    #Deconvolution + Relu (includes dropout) --Layer 13
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Deconvolution + Relu (includes dropout) --Layer 14
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Upsampling --Layer 15
    model.add(UpSampling2D(size=pool_size))

    #Deconvolution + Relu (includes dropout) --Layer 16
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Deconvolution + Relu (includes dropout) --Layer 17
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Deconvolution + Relu (includes dropout) --Layer 18
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    #Upsampling --Layer 19
    model.add(UpSampling2D(size=pool_size))

    #Deconvolution + Relu --Layer 20
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))

    #Deconvolution + Relu -- Final Layer 21 
    model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'relu'))
 
    
    return model

**Load the data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load training images
train_images = pickle.load(open("/content/drive/MyDrive/Deep _learning _project/full_CNN_train.p", "rb" ))
# train_images= tf.image.decode_png(train_images,channels= 3)
# train_images= tf.image.convert_image_dtype(train_images,tf.float32)


# Load image labels
labels = pickle.load(open("/content/drive/MyDrive/Deep _learning _project/full_CNN_labels.p", "rb" ))

# Make into arrays as the neural network wants these
train_images = np.array(train_images)
labels = np.array(labels)
labels = (labels>128).astype('uint8')




(12764, 80, 160, 3)

**see dimensions of the data**

In [ ]:
train_images.shape

(12764, 80, 160, 3)

**Create the train and test splits**

In [ ]:
# Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(train_images, labels)
# Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)


In [ ]:
datagen = ImageDataGenerator(
    #featurewise_center=False
    #zca_whitening=True,
    #width_shift_range=5.0,
    #height_shift_range=5.0,
    brightness_range=[0.2,1],
    channel_shift_range= 20   
)
datagen.fit(X_train)

**Train your model**

In [ ]:
# TODO: Train your model
pool_size = (2, 2)
input_shape = X_train.shape[1:]
batch_size=70
epochs=10
# Create the neural network
model = create_model(input_shape, pool_size)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
#model.fit(X_train, y_train, epochs=6, batch_size=128)
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
    epochs=epochs, verbose=1, validation_data=(X_val, y_val))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/10
164/164 [==============================] - 38s 195ms/step - loss: 0.0440 - accuracy: 0.9422 - val_loss: 0.0175 - val_accuracy: 0.9791
Epoch 2/10
164/164 [==============================] - 29s 179ms/step - loss: 0.0157 - accuracy: 0.9803 - val_loss: 0.0125 - val_accuracy: 0.9840
Epoch 3/10
164/164 [==============================] - 30s 181ms/step - loss: 0.0126 - accuracy: 0.9838 - val_loss: 0.0114 - val_accuracy: 0.9862
Epoch 4/10
164/164 [==============================] - 30s 181ms/step - loss: 0.0114 - accuracy: 0.9851 - val_loss: 0.0100 - val_accuracy: 0.9872
Epoch 5/10
164/164 [==============================] - 30s 180ms/step - loss: 0.0108 - accuracy: 0.9857 - val_loss: 0.0098 - val_accuracy: 0.9875
Epoch 6/10
164/164 [==============================] - 30s 181ms/step - loss: 0.0103 - accuracy: 0.9862 - val_loss: 0.0090 - val_accuracy: 0.9880
Epoch 7/10
164/164 [==============================] - 30s 180ms/step - loss: 0.0100 - accuracy: 0.9865 - val_loss: 0.0088 - val_ac

**Save your model**

In [ ]:
# Save model architecture and weights
model.save('full_CNN_model.h5')

# Show summary of model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 80, 160, 3)       12        
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 78, 158, 8)        224       
                                                                 
 conv2d_1 (Conv2D)           (None, 76, 156, 16)       1168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 38, 78, 16)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 36, 76, 16)        2320      
                                                                 
 dropout (Dropout)           (None, 36, 76, 16)        0